# Creating Your First Graph and Running It in a Session

![9](images/9-1.png)

In [7]:
import tensorflow as tf

print(tf.__version__)

1.8.0


In [1]:
import tensorflow as tf

x = tf.Variable(3, name='x')
y = tf.Variable(4, name='y')
f = x*x*y +y +2

The most important thing to understand is that this code does not actually perform any computation. It just creates a computation graph.

To evaluate this graph, you need to open a TensorFlow session and use it to initialize the variables and evaluate f.

In [3]:
sess = tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
result = sess.run(f)
print(result)

sess.close() # free up resources

42


Having to repeat `sess.run()` all the time is a bit cumbersome(麻烦的), but fortunately there is a better way:

```
with tf.Session() as sess:
    x.initializer().run()
    y.initializer().run()
    result = f.eval()
```

**Error:'Operation' object is not callable.**

In [6]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    init.run()
    result = f.eval()
    print(result)

42


Inside the `with` block, the session is set as the default session.

A TensorFlow program is typically split into two parts:
1. **Construction phase**: builds a computation graph representing the ML model and the computations required to train it.
2. **Execution phase**: runs a lood that evaluates a training setp repeatedly, gradually imporving the model parameters.

# Managing Graphs
Any node you create is automatically to the default graph.

In [8]:
x1 = tf.Variable(1)
x1.graph is tf.get_default_graph()

True

In most cases this is fine, but sometimes you may want to manage multiple independent graphs. You can do this by creating a new `Graph` and temporarily making it the default fraph inside a `with` block.

In [9]:
graph = tf.Graph()

with graph.as_default():
    x2 = tf.Variable(2)
    
x2.graph is graph

True

In [10]:
x2.graph is tf.get_default_graph()

False

In jupyter, it is common to run the same commands more than once. You may end up with a default graph containing many duplicate nodes. One solution is to restart the kernel while another solution is to just reset the default graph by running `tf.reset_default_graph()`.

# Lifecycle of a Node Value
TensorFlow automatically determines the set of nodes that is depends on and it evaluates these nodes first. For example: 
- First, this code defines a very simple graph. 
- Then it starts a session and runs  the graph to evaluate y: TensorFlow automatically detects that y depends on w, which depends on x, so it first evaluates w, then x, then y, and return the value of y.
- Finally, the code runs the graph to evaluate z.

**It is important to note that it will not reuse the result of the previous evaluation of w and x. In short, the code evaluates w and x twice.**

In [11]:
w = tf.constant(3)
x = w+2
y = x+5
z = x*3

with tf.Session() as sess:
    print(y.eval())
    print(z.eval())

10
15


**All node values are dropped between graph runs, except variable values, which are maintained by the session across graph runs. A variable starts its life when its initializer is run, and it ends when the session is closed.**

to Evaluate y and z efficiently, without evaluating w and x twice as in the previous code, ask TensorFlow to evaluate boty y and z in just one graph run.

In [12]:
with tf.Session() as sess:
    y_val,z_val = sess.run([y,z])
    print(y_val)
    print(z_val)

10
15


In single-process TensorFlow, each session would have its own copy of every variable.